In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datetime import datetime
from datetime import timedelta
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import support.ts_class as ts_class
import support.load_and_process_data as lpdata
import math
import gspread
import csv

In [2]:
from importlib import reload
ts_class = reload(ts_class)
lpdata = reload(lpdata)

In [3]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/11/04 15:53:30 WARN Utils: Your hostname, Esas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.10.171 instead (on interface en0)
22/11/04 15:53:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/anaconda3/envs/traiding_spark_delta/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/esak/.ivy2/cache
The jars for the packages stored in: /Users/esak/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ca87255-daab-4ebe-8386-d6d91b1e8c75;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 246ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evict

In [2]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.pandas as ps

ModuleNotFoundError: No module named 'delta'

In [4]:
from numpy import loadtxt

In [5]:
table = "/Users/esak/Projects/stock_traiding/forecast_indices/datasets/stocks_per_minute"
ec_table = "/Users/esak/Projects/stock_traiding/forecast_indices/datasets/economic_data"
table_path = "/Users/esak/Projects/stock_traiding/forecast_indices/datasets/"

22/11/04 17:50:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 300787 ms exceeds timeout 120000 ms
22/11/04 17:50:09 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
msft_2y_trailing = lpdata.av_get_intraday('MSFT',"1min",2,12)

In [ ]:
msft_2y_trailing.isnull().values.any()

In [ ]:
msft_2y_trailing = msft_2y_trailing.reset_index()

In [ ]:
msft_2y_trailing.dtypes

In [ ]:
plt.plot(msft_2y_trailing["Close"])
plt.show()

In [ ]:
TODAY = datetime.today().strftime('%Y-%m-%d')
START_DATE = '1991-01-01 00:00:00'
#START_DATE = '2019-01-01 00:00:00'
END_DATE = TODAY+' 23:59:59'
PERIOD = START_DATE[0:4]+"-"+END_DATE[0:4]

In [ ]:
with open('raw_datasets.txt', 'r') as fd:
    reader = csv.reader(fd)
    for row in reader:
        files = row

In [9]:
tickers = ['SP500','30Y_BOND','10Y_NOTE','GOLD']

In [15]:
for index, file in enumerate(files):
    schema = (StructType()
              .add("Date",StringType(),True)
              .add("Time",StringType(),True)
              .add("Open",DoubleType(),True)
              .add("High",DoubleType(),True)
              .add("Low",DoubleType(),True)
              .add("Close",DoubleType(),True)
              .add("Volume",LongType(),True)
             )
    df = (spark.read.format("csv")
                  .option("header", False)
                  .option("delimiter",";")
                  .schema(schema)
                  .load(file)
                 )
    df = (df
      .withColumn("Datetime", to_timestamp(concat_ws(' ' ,col("Date"),substring(col("Time"), 0, 5)),'dd/MM/yyyy HH:mm'))
      .drop(col("Date"))
      .drop(col("Time"))
      .withColumn("Ticker",lit(tickers[index]))
     )
    df.write.format("delta").mode("overwrite").save(table_path+tickers[index])

22/10/27 15:15:56 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/10/27 15:16:07 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/10/27 16:12:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 287864 ms exceeds timeout 120000 ms
22/10/27 16:12:45 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
df.dtypes

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.write.format("delta").mode("overwrite").save(table)

In [ ]:
msft_2y_trailing.tail()

In [ ]:
cut_datetime = msft_2y_trailing.iloc[0,0]

In [ ]:
str(cut_datetime)

In [ ]:
deltaTable = DeltaTable.forPath(spark, table)

# Declare the predicate by using Spark SQL functions.
deltaTable.delete(col('Datetime') >= str(cut_datetime))

In [ ]:
msft_trailing_spark = ps.from_pandas(msft_2y_trailing).to_spark()

In [ ]:
msft_trailing_spark.show()

In [ ]:
msft_trailing_spark.dtypes

In [ ]:
msft_trailing_spark.write.format("delta").mode("append").save(table)

In [40]:
query = "SELECT * \
FROM delta.`{}` \
ORDER BY Datetime DESC"\
.format(table)

In [41]:
spark.sql(query).show()

+------+------+------+------+------+-------------------+------+
|  Open|  High|   Low| Close|Volume|           Datetime|Ticker|
+------+------+------+------+------+-------------------+------+
| 242.3| 242.3| 242.3| 242.3|  1669|2022-10-21 19:59:00|  MSFT|
|242.22|242.22|242.22|242.22|   186|2022-10-21 19:57:00|  MSFT|
| 242.3| 242.3| 242.3| 242.3|   278|2022-10-21 19:56:00|  MSFT|
|242.41|242.41| 242.4|242.41|  1352|2022-10-21 19:49:00|  MSFT|
|242.41|242.41|242.41|242.41|   307|2022-10-21 19:48:00|  MSFT|
|242.42|242.42|242.42|242.42|   161|2022-10-21 19:44:00|  MSFT|
| 242.4| 242.4| 242.4| 242.4|   252|2022-10-21 19:43:00|  MSFT|
|242.47|242.47|242.47|242.47|   126|2022-10-21 19:41:00|  MSFT|
| 242.3| 242.3| 242.3| 242.3|   394|2022-10-21 19:38:00|  MSFT|
|242.32|242.32|242.32|242.32|   194|2022-10-21 19:36:00|  MSFT|
| 242.3| 242.3| 242.3| 242.3|   231|2022-10-21 19:34:00|  MSFT|
|242.34|242.34| 242.3| 242.3|   585|2022-10-21 19:22:00|  MSFT|
|242.47|242.47|242.47|242.47|   128|2022

In [ ]:
df = spark.read.format("delta").load(table)

In [ ]:
msft_close_np = (np.array(df
                          .filter(col("Datetime") > "2015-01-01")
                          .sort(col("Datetime"))
                          .select(col("Close"))
                          .collect())
                )

In [ ]:
plt.plot(msft_close_np)
plt.show()

In [ ]:
df = df.sort(col("Datetime")).withColumn("Year", year("Datetime"))

In [ ]:
df.show()

In [ ]:
df.write.option("header", True).option("timestampFormat", "yyyy-MM-dd HH:mm:ss").partitionBy("Year").mode("overwrite").csv('data/msft_full')

In [16]:
gc = gspread.service_account(filename='/Users/esak/.config/gspread/service_account.json')

In [54]:
ec_data = lpdata.retrieve_data(["ec_data_1980_2000","ec_2001_2009","ec_2010_2014","ec_2015_2019","ec_2020","ec_2021"])

Retrieving economic data...


In [49]:
ec_data.shape

(66268, 5)

In [55]:
#To correct timezone
ec_data["Datetime"] = ec_data["Datetime"] + timedelta(hours=1)

In [56]:
ec_data.head()

,Datetime,Event,Actual,Forecast,Previous
0,1980-01-01 05:00:00,All Car Sales,1.035000e+07,NaN,9720000.000
1,1980-01-01 05:00:00,All Truck Sales,2.840000e+06,NaN,2940000.000
2,1980-01-01 05:00:00,Capacity Utilization Rate,8.410000e-01,NaN,0.842
3,1980-01-01 05:00:00,CB Consumer Confidence,9.070000e+01,NaN,90.200
4,1980-01-01 05:00:00,CB Employment Trends Index,6.620000e+01,NaN,66.500


In [57]:
ps.from_pandas(ec_data).to_spark().write.format("delta").mode("overwrite").save(ec_table)

22/10/26 20:50:27 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
